### Spuštění PySpark

`export PYSPARK_PYTHON=python3`  
`pyspark --master yarn --num-executors 2 --executor-memory 4G --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, např. `spark.ui.port=10811`

In [ ]:
# uzitecny import
from pyspark.sql import functions as F

### 0. Spark SQL v jednoduchých příkladech

Projděte si soubor s jednoduchými příklady `spark_sql_elementary.ipynb`.

### 1. Načtení dat a zakladní explorace

#### Data

Soubor obsahuje záznamy o písních.

* format: CSV
* cesta: `/user/pascepet/data/lyrics/lyrics.csv` (na HDFS)
* oddelovac: `','`
* header: ne
* pole: `id, nazev, rok, interpret, zanr, text`

1.1 Načtěte soubor do DataFrame (s automatickým odvozením schématu) a přiřaďte sloupcům správné názvy polí (viz výše).

1.2 Soubor nakešujte do paměti.

1.3 Vypište si ukázku dat.

1.4 Zjistěte celkový počet záznamů (řádků) v DataFrame. *(362 237)*

1.5 Zjistěte, kolik písní má jako interpreta Boba Dylana ('bob-dylan'). *(614)*

1.6 Zjistěte, jaky je nejnižší a nejvyšší uvedený rok písně. *(67, 2038)*

1.7 Vypište žánry v pořadí podle počtu zastoupených písní (od největšího po nejmenší). *(Rock 131 377, Pop 49 444 atd.)*

1.8 Vytvořte si z DataFrame dočasnou tabulku a pokuste se zadání 1.4&ndash;1.7 provést i pomocí SQL.

### 2. Úprava a čištění dat

2.1 Vyřaďte všechny záznamy, které mají uvedený rok mimo rozmezí 1950--2018. Zjistěte, kolik záznamů v DataFrame zůstalo. *(362 221)*

2.2 Upravte texty písní takto:  
* Chybějící hodnoty ve sloupci s texty nahraďte prázdnými řetězci.
* Text převeďte na malá písmena.
* Všechny nealfanumerické znaky nahraďte mezerou.
* Posloupnosti více mezer nahraďte jedinou mezerou.
* Vypusťte mezery na obou krajích textu (funkce *trim*).  
Podrobně o regulárních výrazech viz [manuál Pythonu](https://docs.python.org/3.7/library/re.html).

2.3 Přidejte do DataFrame sloupec `slova_poc` obsahující počet všech slov písně. Slova jsou v textu oddělena mezerami. Zkontrolujte, zda pro písně s prázdným polem `text` je počet slov 0, a pokud to tak není, opravte v takových případech počet slov na 0.

2.4 Výsledný DataFrame opět nakešujte.

### 3. Analytické dotazy

3.1 Zjistěte, kolik interpretů ma aspoň 500 písní a kteří to jsou. Vytvořte si pro tyto interprety samostatný DataFrame, využijete ho v zadání 4.3. *(19; Bob Dylan 614, Chris Brown 655 atd.)*

3.2 Vezmeme-li v úvahu jen písně s neprázdným textem (tj. počet slov větší než 0), který z interpretů s aspoň 100 takovými písněmi má nejvyšší průměrný počet slov na píseň? *(eightball-mjg 627,9)*

### 4. Text-mining

4.1 Najděte 20 nejčastěji se vyskytujících aspoň dvouznakových slov v textech písní. (Každé slovo počítejte tolikrát, kolikrát je v textu uvedeno. Zde se hodí DataFrame zpracovat pomocí RDD transformací.)  
*(('the', 2031323), ('you', 1988108), ('to', 1181200), ('and', 1153519), ('it', 910863), ..., ('no', 313002))*  
Poté najděte 20 nejčastěji se vyskytujících aspoň dvouznakových slov s vyloučením stop-words. Soubor se stopwords je na HDFS: `/user/pascepet/data/stopwords.txt`  
*(('can', 354989), ('love', 310137), ('don', 308914), ('know', 291913), ('like', 283256), ..., ('let', 159781))*

4.2 Vyberte z množiny nejčastějších slov mimo stop-words tři podle vlastního uvážení. K DataFrame přidejte tři sloupce (pro každé slovo jeden sloupec) s příznakem True/False, zda je v písni dané slovo aspoň jednou uvedeno.

4.3 U interpretů s aspoň 500 písněmi (viz 3.1) zjistěte, v jakém podílu jejich písní se vyskytují tři vámi vybraná častá slova ze zadání 4.2.

|        interpret| avg(is_like)| avg(is_know)| avg(is_love)|
|-----------------|-------------|-------------|-------------|
|         b-b-king|0.248875565  |0.413793103  |0.40479760   |
| barbra-streisand|0.334935897  |0.384615384  |0.48237179   |
|         bee-gees|0.302170283  |0.472454090  |0.64440734   |
|  atd. | ... | ... | ... |
